# Reference from 
https://www.kaggle.com/code/danpietrow/lgbm-baseline-1-00-lb-beginner-friendly-starter/notebook

And Improving

## Method  
- Train two LGBM models: one for `dx`, one for `dy`  
- Anchor predictions to last observed position (`x_last`, `y_last`)  
- Clip to field bounds for physical plausibility  

# Dependencies

In [ ]:
%%time
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import glob
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Set data directory
DATA_DIR = "/kaggle/input/nfl-big-data-bowl-2026-prediction/"
TRAIN_DIR = os.path.join(DATA_DIR, "train")

# Input and output file patterns
input_files = sorted(glob.glob(os.path.join(DATA_DIR, "train/input_2023_w*.csv")))
output_files = sorted(glob.glob(os.path.join(DATA_DIR, "train/output_2023_w*.csv")))

input_files = pd.concat((pd.read_csv(p) for p in tqdm(input_files, desc="loading inputs")), ignore_index=True)
output_files = pd.concat((pd.read_csv(p) for p in tqdm(output_files, desc="loading outputs")), ignore_index=True)

#
test_in = pd.read_csv(os.path.join(DATA_DIR, "test_input.csv"))
test_template = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
sample_submission = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

print("Inputs:", input_files.shape, "Outputs:", output_files.shape, "Test input:", test_in.shape)

print(f"Found {len(input_files)} input files")
print(f"Found {len(output_files)} output files")   

# Select Final Features for Modeling
Let’s build a clean feature set.

In [ ]:
%%time
feature_columns = [
    # Initial position and motion
    'start_x', 'start_y', 's', 'a', 'vx', 'vy',
    
    # Direction and orientation (radians)
    'dir_rad', 'o_rad',
    
    # Ball landing target (critical!)
    'ball_land_x', 'ball_land_y',
    'dx_ball', 'dy_ball', 'dist_to_ball', 'angle_to_ball', 'alignment_to_ball',
    
    # Temporal progression in air
    'frame_id_output', 'frame_norm', 'frame_sq',
    
    # Player physical traits
    'player_weight', 'player_height', 'age',
    
    # Role and position flags
    'is_target', 'is_defender', 'is_WR', 'is_CB', 'is_TE', 'is_RB', 'is_LB', 'is_S', 'is_DL',
    
    # Play context
    'absolute_yardline_number',
    'play_direction',  # may need encoding

   # CAT_FEATS = ['player_role','player_side','play_direction']
   # TARGETS = ['dx','dy']
]   

# Feature Engineering

In [ ]:
%%time
# Feature Engineering
import pandas as pd
import numpy as np

def prepare_last_obs(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the last observed position of each player before the pass.
    Convert player height from 'feet-inches' format (e.g., '5-11') to inches.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input tracking data with columns: game_id, play_id, nfl_id, frame_id, x, y, etc.
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with one row per player per play, containing their last observed state.
    """
    # Sort once, then group (assumes frame_id increases with time)
    df_sorted = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    
    # Get last observation per player per play
    df_last = (df_sorted
               .groupby(['game_id', 'play_id', 'nfl_id'], as_index=False)
               .last())  # This grabs last non-null values across all columns

    # Rename spatial coordinates
    df_last = df_last.rename(columns={'x': 'x_last', 'y': 'y_last'})
    
    # Vectorized height conversion: convert '5-11' → 71
    def height_str_to_inches(height_series: pd.Series) -> pd.Series:
        # Split on '-' and handle non-string or missing values
        split_height = height_series.astype(str).str.split('-', expand=True)
        feet = pd.to_numeric(split_height[0], errors='coerce')
        inches = pd.to_numeric(split_height[1], errors='coerce')
        return feet * 12 + inches

    df_last['player_height'] = height_str_to_inches(df_last['player_height'])
    
    return df_last


def add_target_info(df_last: pd.DataFrame) -> pd.DataFrame:
    """
    Add targeted receiver's last position to all players in the same play.
    
    Parameters:
    -----------
    df_last : pd.DataFrame
        Player-level data with last observed positions.
    
    Returns:
    --------
    pd.DataFrame
        Same data, but with target_last_x, target_last_y, and target_nfl_id for each play.
    """
    # Extract targeted receiver info
    targets = (df_last[df_last['player_role'] == 'Targeted Receiver']
               [['game_id', 'play_id', 'nfl_id', 'x_last', 'y_last']]
               .copy())
    
    targets.rename(columns={
        'nfl_id': 'target_nfl_id',
        'x_last': 'target_last_x',
        'y_last': 'target_last_y'
    }, inplace=True)
    
    # Merge target info back to all players in the play
    df_enhanced = df_last.merge(
        targets[['game_id', 'play_id', 'target_last_x', 'target_last_y', 'target_nfl_id']],
        on=['game_id', 'play_id'],
        how='left'
    )
    
    return df_enhanced


def create_features(df: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
    """
    Create derived features for modeling.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Merged dataset with last positions and target info.
    is_train : bool
        Whether to compute target variables (dx, dy).

    Returns:
    --------
    pd.DataFrame
        Dataset with engineered features.
    """
    # Avoid modifying the original DataFrame
    df = df.copy()

    # Time-related features (frame and time since last observation)
    df['frame_offset'] = df['frame_id']
    df['time_offset'] = df['frame_offset'] / 10.0  # Assuming 10 Hz sampling rate

    # Distance from player's last position to ball landing point
    df['dist_to_ball_land'] = np.sqrt(
        (df['ball_land_x'] - df['x_last'])**2 + 
        (df['ball_land_y'] - df['y_last'])**2
    )

    # Angle from player to ball landing point (in radians)
    df['angle_to_ball_land'] = np.arctan2(
        df['ball_land_y'] - df['y_last'],
        df['ball_land_x'] - df['x_last']
    )

    # Distance from player to targeted receiver's last position
    df['dist_to_target_last'] = np.sqrt(
        (df['target_last_x'] - df['x_last'])**2 + 
        (df['target_last_y'] - df['y_last'])**2
    )

    # Binary flag: is this player the targeted receiver?
    df['is_target'] = (df['nfl_id'] == df['target_nfl_id']).astype(int)

    # Optional: additional useful features (uncomment if needed)
    # df['speed_to_ball'] = df['dist_to_ball_land'] / (df['time_offset'] + 1e-6)  # avoid div/0
    # df['relative_dir'] = (df['dir'] - df['angle_to_ball_land']) % (2 * np.pi)

    # Only compute target deltas in training mode
    if is_train:
        df['dx'] = df['x'] - df['x_last']  # change in x from last observed
        df['dy'] = df['y'] - df['y_last']  # change in y from last observed

    return df    
    
def prepare_train(df_in: pd.DataFrame, df_out: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare the training dataset by:
    - Extracting last observed player state before the pass
    - Adding targeted receiver position to all players in the play
    - Merging with future positions (targets)
    - Engineering motion and spatial features

    Parameters:
    ----------
    df_in : pd.DataFrame
        Tracking data up to the moment of pass (used to get last observation).
        Must include: game_id, play_id, nfl_id, frame_id, x, y, s, a, o, dir,
                      player_height, player_weight, player_role, etc.
    df_out : pd.DataFrame
        Future positions to predict (output labels).
        Must include: game_id, play_id, nfl_id, frame_id, x, y

    Returns:
    -------
    pd.DataFrame
        Training-ready DataFrame with features and target deltas (dx, dy).
    """
    # Validate inputs
    required_in_cols = {'game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y'}
    required_out_cols = {'game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y'}

    assert required_in_cols.issubset(df_in.columns), f"Missing columns in df_in: {required_in_cols - set(df_in.columns)}"
    assert required_out_cols.issubset(df_out.columns), f"Missing columns in df_out: {required_out_cols - set(df_out.columns)}"

    # Step 1: Get last observed state per player per play
    last_obs = prepare_last_obs(df_in)

    # Step 2: Add targeted receiver's last position to all players in the same play
    last_obs = add_target_info(last_obs)

    # Define columns to carry forward (explicit list for clarity and performance)
    cols_to_keep = [
        'game_id', 'play_id', 'nfl_id',
        'x_last', 'y_last', 's', 'a', 'o', 'dir',
        'player_role', 'player_side', 'num_frames_output',
        'ball_land_x', 'ball_land_y',
        'target_last_x', 'target_last_y', 'target_nfl_id',
        'play_direction', 'absolute_yardline_number',
        'player_height', 'player_weight'
    ]

    # Ensure we only select existing columns (defensive programming)
    available_cols = [c for c in cols_to_keep if c in last_obs.columns]
    missing_cols = set(cols_to_keep) - set(available_cols)
    if missing_cols:
        print(f"[Warning] Missing columns in last_obs, skipping: {missing_cols}")

    # Step 3: Merge with future positions (labels)
    train = df_out.merge(
        last_obs[available_cols],
        on=['game_id', 'play_id', 'nfl_id'],
        how='left',
        suffixes=('', '_drop')
    )

    # Drop any duplicate columns (in case of suffix issues)
    train = train.loc[:, ~train.columns.duplicated()]

    # Step 4: Feature engineering
    train = create_features(train, is_train=True)

    # Optional: Drop rows where last observation is missing (e.g., new players not in df_in)
    initial_len = len(train)
    train.dropna(subset=['x_last', 'y_last'], inplace=True)
    final_len = len(train)
    if final_len < initial_len:
        print(f"[Info] Dropped {initial_len - final_len} rows due to missing last observation.")

    return train
def prepare_test(test_in: pd.DataFrame, test_template: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare the test dataset using the same feature engineering pipeline as training.
    Does not compute target variables (dx, dy).

    Parameters:
    ----------
    test_in : pd.DataFrame
        Tracking data up to the pass moment (last observed player states).
        Must include: game_id, play_id, nfl_id, frame_id, x, y, s, a, o, dir, etc.
    test_template : pd.DataFrame
        Template of future frames to predict (provided by competition or pipeline).
        Must include: game_id, play_id, nfl_id, frame_id

    Returns:
    -------
    pd.DataFrame
        Test-ready DataFrame with features, ready for model inference.
    """
    # Validate inputs
    required_in_cols = {'game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y'}
    required_template_cols = {'game_id', 'play_id', 'nfl_id', 'frame_id'}

    assert required_in_cols.issubset(test_in.columns), (
        f"Missing required columns in test_in: {required_in_cols - set(test_in.columns)}"
    )
    assert required_template_cols.issubset(test_template.columns), (
        f"Missing required columns in test_template: {required_template_cols - set(test_template.columns)}"
    )

    # Step 1: Get last observed state per player per play
    last_test = prepare_last_obs(test_in)

    # Step 2: Add targeted receiver info to all players in the play
    last_test = add_target_info(last_test)

    # Define feature columns (same as in training for consistency)
    cols_to_keep = [
        'game_id', 'play_id', 'nfl_id',
        'x_last', 'y_last', 's', 'a', 'o', 'dir',
        'player_role', 'player_side', 'num_frames_output',
        'ball_land_x', 'ball_land_y',
        'target_last_x', 'target_last_y', 'target_nfl_id',
        'play_direction', 'absolute_yardline_number',
        'player_height', 'player_weight'
    ]

    # Select only available columns (defensive: in case some metadata is missing)
    available_cols = [c for c in cols_to_keep if c in last_test.columns]
    missing_cols = set(cols_to_keep) - set(available_cols)
    if missing_cols:
        print(f"[Warning] Missing in test last_obs, skipping: {missing_cols}")

    # Step 3: Merge with test template (future frames to predict)
    test_rows = test_template.merge(
        last_test[available_cols],
        on=['game_id', 'play_id', 'nfl_id'],
        how='left',
        suffixes=('', '_drop')
    )

    # Drop duplicated columns (in case of merge artifacts)
    test_rows = test_rows.loc[:, ~test_rows.columns.duplicated()]

    # Step 4: Create features (no dx/dy since no true 'x', 'y' in test output)
    test_rows = create_features(test_rows, is_train=False)

    # Optional: Warn about missing last observations
    missing_last = test_rows[['x_last', 'y_last']].isna().any(axis=1).sum()
    if missing_last > 0:
        print(f"[Warning] {missing_last} rows in test have no last observed position. Filled with NaN.")

    return test_rows   

In [ ]:
%%time
# trying
# Prepare datasets
train = prepare_train(input_files, output_files)
test = prepare_test(test_in, test_template)

# Select features to use 
FEATURES = [
    'x_last','y_last','s','a','o','dir',
    'frame_offset','time_offset',
    'dist_to_ball_land','angle_to_ball_land',
    'dist_to_target_last','is_target',
    'absolute_yardline_number', 'player_height', 'player_weight'
]
CAT_FEATS = ['player_role','player_side','play_direction']
TARGETS = ['dx','dy']

# EDA

 ### Peek at Inputs & Outputs

In [ ]:
%%time
print("=== INPUT COLUMNS (Sample) ===")
print(input_files.columns.tolist()[:15], "...")

print("\n=== OUTPUT COLUMNS ===")
print(output_files.columns.tolist())

print("\n=== TEST INPUT SHAPE ===")
print(test_in.shape)   

In [ ]:
%%time
# Flag which rows are to be predicted
target_mask = input_files['player_to_predict'] == 1

print(f"Total rows: {len(input_files):,}")
print(f"Target rows (player_to_predict == 1): {target_mask.sum():,}")
print(f"Target fraction: {target_mask.mean():.1%}")   

### 📊 Core EDA 1: Target Distribution (if available)

In [ ]:
%%time
targets = input_files[target_mask].copy()

plt.figure(figsize=(10, 4))
role_counts = targets['player_role'].value_counts()
sns.barplot(x=role_counts.index, y=role_counts.values, palette='coolwarm')
plt.title("Player Roles Being Predicted | Core EDA = 1.00", fontsize=14, weight='bold')
plt.ylabel("Count")
plt.xlabel("Player Role")
plt.xticks(rotation=30)
for i, v in enumerate(role_counts.values):
    plt.text(i, v + 50, str(v), ha='center', va='bottom', fontsize=9)
plt.grid(axis='y', alpha=0.3)
plt.show()

print("Top 5 player roles to predict:\n", role_counts.head())   

### 📊 Core EDA 2: Input vs Output Position (x, y)

In [ ]:
%%time
# Create merge key
input_files['key'] = input_files[['game_id', 'play_id', 'nfl_id', 'frame_id']].astype(str).agg('-'.join, axis=1)
output_files['key'] = output_files[['game_id', 'play_id', 'nfl_id', 'frame_id']].astype(str).agg('-'.join, axis=1)   

Now merge input (with x, y) to output (true x, y) on key:

In [ ]:
%%time
# Merge input and output on key
merged_io = input_files[['key', 'x', 'y', 's', 'a', 'player_position', 'player_role', 'play_direction']].copy()
merged_io = merged_io.rename(columns={'x': 'input_x', 'y': 'input_y'})

merged_io = merged_io.merge(
    output_files[['key', 'x', 'y']],
    on='key',
    how='inner'
).rename(columns={'x': 'target_x', 'y': 'target_y'})

print(f"Merged dataset (input vs target): {len(merged_io):,} frames")   

If merge fails → maybe nfl_id missing or type mismatch. Let's ensure dtypes match.

In [ ]:
%%time
for col in ['game_id', 'play_id', 'nfl_id', 'frame_id']:
    input_files[col] = input_files[col].astype(str)
    output_files[col] = output_files[col].astype(str)   

Then recompute key and merge.

### 📊 Core EDA 3: Position Error Distribution (Input vs Target)
Now compute the raw displacement error in yards:

In [ ]:
%%time
merged_io['error_x'] = merged_io['target_x'] - merged_io['input_x']
merged_io['error_y'] = merged_io['target_y'] - merged_io['input_y']
merged_io['error_dist'] = np.sqrt(merged_io['error_x']**2 + merged_io['error_y']**2)

print(f"Mean absolute position error: {merged_io['error_dist'].mean():.3f} yards")
print(f"Median error: {merged_io['error_dist'].median():.3f} yards")
print(f"95th percentile error: {merged_io['error_dist'].quantile(0.95):.3f} yards")   

In [ ]:
### 📈 Plot: Error Distribution
plt.figure(figsize=(10, 4))
sns.histplot(merged_io['error_dist'], bins=100, kde=True, color='purple')
plt.title("Input vs Target Position Error | Core EDA = 1.00", fontsize=14, weight='bold')
plt.xlabel("Position Error (yards)")
plt.ylabel("Density")
plt.axvline(merged_io['error_dist'].mean(), color='red', linestyle='--', label=f"Mean: {merged_io['error_dist'].mean():.3f}")
plt.legend()
plt.grid(alpha=0.2)
plt.xlim(0, merged_io['error_dist'].quantile(0.99))
plt.show()  

### 📊 Core EDA 4: Frame Density & Temporal Resolution
Check how many frames per play:

In [ ]:
%%time
frames_per_play = input_files.groupby(['game_id', 'play_id']).size()
plt.figure(figsize=(10, 4))
sns.histplot(frames_per_play, bins=50, kde=False, color='teal')
plt.title("Frames per Play | Core EDA = 1.00", fontsize=14, weight='bold')
plt.xlabel("Number of Frames")
plt.ylabel("Play Count")
plt.axvline(frames_per_play.median(), color='red', linestyle='--', label=f"Median: {frames_per_play.median()}")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

print(f"Median frames per play: {frames_per_play.median()}")
print(f"Mean frames per play: {frames_per_play.mean():.1f}")   

# Train Model

In [ ]:
%%time
def train_directional_models(
    train: pd.DataFrame,
    features: list,
    cat_features: list,
    target_dx: str = 'dx',
    target_dy: str = 'dy',
    lgbm_params: dict = None,
    valid_size: float = 0.1,
    use_early_stopping: bool = True,
    verbose: int = 20
) -> dict:
    """
    Train two LightGBM models for dx and dy with proper early stopping support.
    """
    import lightgbm as lgb
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error
    import numpy as np

    # Default parameters
    if lgbm_params is None:
        lgbm_params = {
            'objective': 'regression',
            'boosting_type': 'gbdt',
            'n_estimators': 1000,
            'learning_rate': 0.1,
            'num_leaves': 32,
            'max_depth': -1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'random_state': 42,
            'verbosity': -1
        }

    # Extract features
    X = train[features].copy()
    for col in cat_features:
        if col in X.columns:
            X[col] = X[col].astype('category')
        else:
            raise ValueError(f"Categorical feature '{col}' not found.")

    y_dx = train[target_dx].values
    y_dy = train[target_dy].values

    # Split data for early stopping
    if use_early_stopping:
        X_train, X_val, y_train_dx, y_val_dx = train_test_split(
            X, y_dx, test_size=valid_size, random_state=42
        )
        _, _, y_train_dy, y_val_dy = train_test_split(
            X, y_dy, test_size=valid_size, random_state=42
        )
        print(f"Using train/val split for early stopping (val_size={valid_size})")
    else:
        X_train, y_train_dx, y_train_dy = X, y_dx, y_dy
        X_val = y_val_dx = y_val_dy = None
        print("Training without validation split or early stopping.")

    # Train model for dx
    print("→ Training model for dx...")
    model_dx = LGBMRegressor(**lgbm_params)

    fit_args_dx = {
        'X': X_train,
        'y': y_train_dx,
        'eval_set': [(X_val, y_val_dx)] if use_early_stopping else None,
        'eval_names': ['val'] if use_early_stopping else None,
        'categorical_feature': cat_features,
        #'early_stopping_rounds': 50 if use_early_stopping else None,
        #'verbose': verbose if use_early_stopping else -1
    }

    # Only pass early_stopping_rounds if eval_set is provided
    #f not use_early_stopping:
    #    fit_args_dx.pop('early_stopping_rounds')
    #    fit_args_dx.pop('eval_names')

    model_dx.fit(**fit_args_dx)

    # Train model for dy
    print("→ Training model for dy...")
    model_dy = LGBMRegressor(**lgbm_params)

    fit_args_dy = {
        'X': X_train,
        'y': y_train_dy,
        'eval_set': [(X_val, y_val_dy)] if use_early_stopping else None,
        'eval_names': ['val'] if use_early_stopping else None,
        'categorical_feature': cat_features
    }

    model_dy.fit(**fit_args_dy)   
    val_rmse = (0.0, 0.0)
    if use_early_stopping:
        val_pred_dx = model_dx.predict(X_val)
        val_pred_dy = model_dy.predict(X_val)

        rmse_dx = np.sqrt(mean_squared_error(y_val_dx, val_pred_dx))
        rmse_dy = np.sqrt(mean_squared_error(y_val_dy, val_pred_dy))
        val_rmse = (rmse_dx, rmse_dy)

        print(f"✅ Final RMSE | dx: {rmse_dx:.4f}, dy: {rmse_dy:.4f}") 
        # Feature importance
        fi = pd.DataFrame({
        'feature': model_dx.feature_name_,
        'importance': model_dx.feature_importances_
    }).sort_values('importance', ascending=False)

    print("→ Top 10 most important features:")
    print(fi.head(10))

    return {
        'model_dx': model_dx,
        'model_dy': model_dy,
        'val_rmse': val_rmse,
        'feature_importance': fi
    }   

In [ ]:
%%time
models = train_directional_models(
    train=train,
    features=FEATURES + CAT_FEATS,
    cat_features=CAT_FEATS,
    valid_size=0.1
)

model_dx = models['model_dx']
model_dy = models['model_dy']   

# Prediction

In [ ]:
%%time
# === PREDICTION: Reconstruct Future (x, y) with Safety Checks ===

# Define features
PREDICTION_FEATURES = FEATURES + CAT_FEATS

# Safety: Check if all features are present
missing_feats = [f for f in PREDICTION_FEATURES if f not in test.columns]
if missing_feats:
    raise KeyError(f"Missing features in test data: {missing_feats}")

# Create feature matrix
X_test = test[PREDICTION_FEATURES].copy()

# Handle categorical features safely
for col in CAT_FEATS:
    if col in X_test.columns:
        # Fill NaN to avoid category conversion error
        X_test[col] = X_test[col].fillna("missing").astype("category")
    else:
        print(f"⚠️ Categorical feature '{col}' not found. Skipping.")

# Initialize predictions
print(f"Predicting on {len(X_test)} rows...")
pred_dx = np.zeros(len(X_test), dtype=np.float32)
pred_dy = np.zeros(len(X_test), dtype=np.float32)

# Predict Δx and Δy
try:
    pred_dx += model_dx.predict(X_test)
    pred_dy += model_dy.predict(X_test)
    print("✅ Prediction complete.")
except Exception as e:
    raise RuntimeError(f"Prediction failed: {e}")

# Reconstruct absolute coordinates
# Safety: Ensure x_last and y_last exist
for coord in ['x_last', 'y_last']:
    if coord not in test.columns:
        raise KeyError(f"Required column '{coord}' not found in test.")

test['pred_x'] = test['x_last'].values + pred_dx
test['pred_y'] = test['y_last'].values + pred_dy

# Clip to field boundaries (optional but smart)
test['pred_x'] = test['pred_x'].clip(0, 120)  # including end zones
test['pred_y'] = test['pred_y'].clip(0, 53.3)  # standard field width

print(f"📍 Final prediction range: x=({test['pred_x'].min():.2f}, {test['pred_x'].max():.2f}), "
      f"y=({test['pred_y'].min():.2f}, {test['pred_y'].max():.2f})")   

In [ ]:
%%time
def make_predictions(
    test: pd.DataFrame,
    features: list,
    cat_features: list,
    model_dx,
    model_dy,
    x_last_col: str = 'x_last',
    y_last_col: str = 'y_last',
    max_speed_per_sec: float = 10.0  # yards per second (reasonable max sprint)
) -> pd.DataFrame:
    """
    Generate future position predictions using trained models.
    
    Applies:
    - Categorical type conversion
    - Model inference
    - Movement validation (clip unrealistic deltas)
    - Safe assignment to copy of input
    
    Parameters:
    -----------
    test : pd.DataFrame
        Test data with features and last observed positions.
    features : list
        List of feature names used in training.
    cat_features : list
        Names of categorical features.
    model_dx, model_dy : trained models
        Models predicting displacement in x and y.
    x_last_col, y_last_col : str
        Column names for last known x/y.
    max_speed_per_sec : float
        Maximum allowed speed to clip implausible predictions.

    Returns:
    --------
    pd.DataFrame
        Copy of `test` with 'pred_x' and 'pred_y' added.
    """
    # Input validation
    missing_features = [f for f in features if f not in test.columns]
    if missing_features:
        raise ValueError(f"Missing features in test data: {missing_features}")

    # Create a clean copy to avoid SettingWithCopyWarning
    df = test.copy()

    # Prepare feature matrix
    X_test = df[features].copy()

    # Convert categorical columns
    for col in cat_features:
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')
        else:
            raise ValueError(f"Categorical feature '{col}' not found in test set.")

    # Predict deltas
    try:
        pred_dx = model_dx.predict(X_test)
        pred_dy = model_dy.predict(X_test)
    except Exception as e:
        raise RuntimeError(f"Model prediction failed: {e}")

    # --- Optional: Clip unrealistic movements ---
    # Use time_offset (in seconds) to compute max allowed displacement
    if 'time_offset' in df.columns:
        time_seconds = df['time_offset'].values  # e.g., frame_id / 10.0
        time_seconds = np.maximum(time_seconds, 0.1)  # avoid div/0 or instant jumps

        max_speed_per_frame = max_speed_per_sec * time_seconds
        max_displacement = max_speed_per_frame  # in yards

        total_pred_dist = np.sqrt(pred_dx**2 + pred_dy**2)

        scale = np.where(
            total_pred_dist > max_displacement,
            max_displacement / (total_pred_dist + 1e-8),
            1.0
        )
        pred_dx = pred_dx * scale
        pred_dy = pred_dy * scale

        clipped_count = (scale < 1.0).sum()
        if clipped_count > 0:
            print(f"[!] Clipped {clipped_count} predictions for excessive speed.")

    # Reconstruct absolute positions
    pred_x = df[x_last_col].values + pred_dx
    pred_y = df[y_last_col].values + pred_dy

    # Assign predictions
    df['pred_x'] = pred_x
    df['pred_y'] = pred_y

    # Optional: Add raw deltas for debugging
    # df['pred_dx'] = pred_dx
    # df['pred_dy'] = pred_dy

    print(f"✅ Predictions generated: {len(df):,} rows")
    return df


# --- Usage ---
test_pred = make_predictions(
    test=test,
    features=FEATURES + CAT_FEATS,
    cat_features=CAT_FEATS,
    model_dx=model_dx,
    model_dy=model_dy,
    max_speed_per_sec=12.0  # elite player sprint speed (~27 mph ≈ 12 yd/sec)
)   

# Create submission

In [ ]:
%%time
def create_submission(
    df: pd.DataFrame,
    id_columns: list = ['game_id', 'play_id', 'nfl_id', 'frame_id'],
    pred_x_col: str = 'pred_x',
    pred_y_col: str = 'pred_y',
    output_path: str = "submission.csv"
) -> pd.DataFrame:
    """
    Create a submission CSV by:
    - Generating a unique 'id' from key columns
    - Selecting and renaming predicted coordinates
    - Saving to CSV

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing predictions and ID components.
    id_columns : list
        List of column names to join with underscore for 'id'.
    pred_x_col : str
        Name of predicted x column.
    pred_y_col : str
        Name of predicted y column.
    output_path : str
        Path to save the submission CSV.

    Returns:
    --------
    pd.DataFrame
        Submission-ready DataFrame with 'id', 'x', 'y'.
    """
    required_cols = id_columns + [pred_x_col, pred_y_col]
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        raise ValueError(f"Missing required columns in DataFrame: {missing_cols}")

    # Ensure all ID components are string type (avoid 'NaN' or float issues)
    id_components = df[id_columns].astype(str)
    
    # Vectorized: create 'id' using underscore join across columns
    submission = df.copy()
    submission['id'] = id_components[id_columns[0]]
    for col in id_columns[1:]:
        submission['id'] = submission['id'] + "_" + id_components[col]
    
    # Select and rename prediction columns
    submission = submission[['id', pred_x_col, pred_y_col]].rename(
        columns={pred_x_col: 'x', pred_y_col: 'y'}
    )
    
    # Optional: Validate no missing values in submission
    if submission.isnull().any().any():
        null_counts = submission.isnull().sum()
        print(f"[Warning] Null values found in submission:\n{null_counts[null_counts > 0]}")
    
    # Save to CSV
    submission.to_csv(output_path, index=False)
    print(f"✅ Submission saved to '{output_path}' | Shape: {submission.shape}")
    
    return submission


# Usage:
submission = create_submission(
    df=test,
    id_columns=['game_id', 'play_id', 'nfl_id', 'frame_id'],
    pred_x_col='pred_x',
    pred_y_col='pred_y',
    output_path="submission.csv"
)

# Display preview
submission.head()   